In [1]:
%matplotlib inline

import matplotlib 
import numpy as np
import matplotlib.pyplot as plt

In [1]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
udev            7.9G   12K  7.9G   1% /dev
tmpfs           1.6G  404K  1.6G   1% /run
/dev/vda1       158G   24G  128G  16% /
none            4.0K     0  4.0K   0% /sys/fs/cgroup
none            5.0M     0  5.0M   0% /run/lock
none            7.9G     0  7.9G   0% /run/shm
none            100M   20K  100M   1% /run/user
/dev/vda15      105M  3.2M  102M   4% /boot/efi


In [2]:
import pandas as pd
import json
import re
pd.options.display.max_rows = 200

In [5]:
df_review = pd.read_pickle('yelp_review_cleaned_0710.pkl')

In [6]:
df_review.head(10)

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,lemma_clean,sent_count
0,0W4lkclzZThpx3V65bVgig,0,2016-05-28,0,v0i_UHJMo_hPBq9bxWvW4w,5,"Love the staff, love the meat, love the place....",0,bv2nCi5Qv5vroFiqKGopiw,"[love, the, staff, love, the, meat, love, the,...",6
1,AEx2SYEUJmTxVVB18LlCwA,0,2016-05-28,0,vkVSCC7xljjrAI4UGfnKEQ,5,Super simple place but amazing nonetheless. It...,0,bv2nCi5Qv5vroFiqKGopiw,"[super, simple, place, but, amazing, nonethele...",3
2,VR6GpWIda3SfvPC-lg9H3w,0,2016-05-28,0,n6QzIUObkYshz4dz2QRJTw,5,Small unassuming place that changes their menu...,0,bv2nCi5Qv5vroFiqKGopiw,"[small, unassum, place, that, change, PRON, me...",8
3,CKC0-MOWMqoeWf6s-szl8g,0,2016-05-28,0,MV3CcKScW05u5LVfF6ok0g,5,Lester's is located in a beautiful neighborhoo...,0,bv2nCi5Qv5vroFiqKGopiw,"[lester, s, be, locate, in, a, beautiful, neig...",4
4,ACFtxLv8pGrrxMm6EgjreA,0,2016-05-28,0,IXvOzsEMYtiJI0CARmj77Q,4,Love coming here. Yes the place always needs t...,0,bv2nCi5Qv5vroFiqKGopiw,"[love, come, here, yes, the, place, always, ne...",6
5,s2I_Ni76bjJNK9yG60iD-Q,0,2016-05-28,0,L_9BTb55X0GDtThi6GlZ6w,4,Had their chocolate almond croissant and it wa...,0,bv2nCi5Qv5vroFiqKGopiw,"[have, PRON, chocolate, almond, croissant, and...",5
6,8QWPlVQ6D-OExqXoaD2Z1g,0,2014-09-24,0,HRPm3vEZ_F-33TYVT7Pebw,5,Cycle Pub Las Vegas was a blast! Got a groupon...,1,_4iMDXbXZ1p1ONG297YEAQ,"[cycle, pub, las, vegas, be, a, blast, get, a,...",8
7,9_CGhHMz8698M9-PkVf0CQ,2,2012-05-11,0,ymAUG8DZfQcFTBSOiaNN4w,4,Who would have guess that you would be able to...,0,u0LXt3Uea_GidxRW1xcsfg,"[who, would, have, guess, that, PRON, would, b...",6
8,gkCorLgPyQLsptTHalL61g,0,2015-10-27,0,8UIishPUD92hXtScSga_gw,4,Always drove past this coffee house and wonder...,1,u0LXt3Uea_GidxRW1xcsfg,"[always, drive, past, this, coffee, house, and...",5
9,5r6-G9C4YLbC7Ziz57l3rQ,0,2013-02-09,0,w41ZS9shepfO3uEyhXEWuQ,3,"Not bad!! Love that there is a gluten-free, ve...",1,u0LXt3Uea_GidxRW1xcsfg,"[not, bad, love, that, there, be, a, gluten, f...",5


In [109]:
import spacy
nlp = spacy.load('en')

In [89]:
aaa = df_review['lemma'][0]
text1 = [' '.join(aaa)]

In [7]:
### each row in the lemma column is a list of words. Now converting it back 
### to text/str, so CountVectorizor has one row for each review
df_review['lemmatized'] = map(lambda x: [' '.join(x)],df_review['lemma_clean'])

In [8]:
df_review.to_pickle('yelp_review_cleaned_0710v2.pkl')

In [9]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
business_id    100000 non-null object
cool           100000 non-null int64
date           100000 non-null datetime64[ns]
funny          100000 non-null int64
review_id      100000 non-null object
stars          100000 non-null int64
text           100000 non-null object
useful         100000 non-null int64
user_id        100000 non-null object
lemma_clean    100000 non-null object
sent_count     100000 non-null int64
lemmatized     100000 non-null object
dtypes: datetime64[ns](1), int64(5), object(6)
memory usage: 9.2+ MB


## Get all the words and counts for each review

In [13]:
### Run CountVectorizer for each row in ['lemmatized']
from sklearn.feature_extraction.text import CountVectorizer

lemma_count = CountVectorizer()
text1 = df_review['lemmatized'][0]
# text1 = ['happy','birthday','new','york','city','happy']
# text1 = ['happy birthday new york city happy']

counts = lemma_count.fit_transform(text1) # a sparse matrix
counts = counts.toarray()
# counts.sum(axis=0) # same as toarray, but it's a matrix
# print counts
# print counts.shape
vocab = lemma_count.get_feature_names() # a list
# len(vocab)


In [40]:
counts.reshape(40,)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3,
       1, 1, 2, 2, 2, 1, 1, 1, 1, 7, 1, 1, 1, 1, 1, 3, 2, 2])

In [53]:
vv = np.array(vocab)
vv = vv.reshape(-1,)
counts = counts.reshape(-1,)
zip(vv, counts)
# a = np.array([1,2,3])
# a_temp = np.array([1,2,3]).reshape(3,1)
# print a.shape
# print a_temp.shape

[(u'and', 1),
 (u'around', 1),
 (u'ask', 1),
 (u'can', 1),
 (u'cut', 1),
 (u'dinner', 1),
 (u'do', 1),
 (u'fatty', 1),
 (u'for', 1),
 (u'french', 1),
 (u'fry', 1),
 (u'get', 1),
 (u'half', 1),
 (u'hand', 1),
 (u'hot', 1),
 (u'hour', 1),
 (u'how', 1),
 (u'just', 1),
 (u'lean', 1),
 (u'line', 1),
 (u'long', 1),
 (u'love', 3),
 (u'lunch', 1),
 (u'maybe', 1),
 (u'meat', 2),
 (u'not', 2),
 (u'or', 2),
 (u'pepper', 1),
 (u'pickle', 1),
 (u'place', 1),
 (u'prepare', 1),
 (u'pron', 7),
 (u'remember', 1),
 (u'say', 1),
 (u'something', 1),
 (u'sour', 1),
 (u'staff', 1),
 (u'the', 3),
 (u'too', 2),
 (u'want', 2)]

In [14]:
def count_lemma(text):
    lemma_count = CountVectorizer(stop_words = None, analyzer = 'word', token_pattern= r"(?u)\b\w+\b")
    counts = lemma_count.fit_transform(text)
    vocab = lemma_count.get_feature_names()
    vocab = np.array(vocab).reshape(-1,)
    counts = counts.toarray().reshape(-1,)
            
    return zip(vocab, counts) 


# testing = df_review['lemmatized'][:5]
# df_review['testing'] = None

# test = CountVectorizer()
# for i,text in enumerate(testing):
#     df_review['testing'][i] = count_lemma(text)

# df_review['testing'][0]
    

In [148]:
def count_lemma_test(text):
    lemma_count = CountVectorizer(stop_words = None, analyzer = 'word', token_pattern= r"[^\d\W]")
    counts = lemma_count.fit_transform(text)
    vocab = lemma_count.get_feature_names()
    vocab = np.array(vocab).reshape(-1,)
    counts = counts.toarray().reshape(-1,)
            
    return zip(vocab, counts) 

In [15]:
df_review['count_vec'] = map(lambda x: count_lemma(x), df_review['lemmatized'])


In [16]:
df_review.to_pickle('yelp_review_cleaned_0710v3.pkl')

In [18]:
df_review.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,lemma_clean,sent_count,lemmatized,count_vec
0,0W4lkclzZThpx3V65bVgig,0,2016-05-28,0,v0i_UHJMo_hPBq9bxWvW4w,5,"Love the staff, love the meat, love the place....",0,bv2nCi5Qv5vroFiqKGopiw,"[love, the, staff, love, the, meat, love, the,...",6,[love the staff love the meat love the place p...,"[(a, 3), (and, 1), (around, 1), (ask, 1), (can..."
1,AEx2SYEUJmTxVVB18LlCwA,0,2016-05-28,0,vkVSCC7xljjrAI4UGfnKEQ,5,Super simple place but amazing nonetheless. It...,0,bv2nCi5Qv5vroFiqKGopiw,"[super, simple, place, but, amazing, nonethele...",3,[super simple place but amazing nonetheless PR...,"[(30, 1), (a, 1), (amazing, 1), (and, 3), (aro..."
2,VR6GpWIda3SfvPC-lg9H3w,0,2016-05-28,0,n6QzIUObkYshz4dz2QRJTw,5,Small unassuming place that changes their menu...,0,bv2nCi5Qv5vroFiqKGopiw,"[small, unassum, place, that, change, PRON, me...",8,[small unassum place that change PRON menu eve...,"[(10, 1), (30, 1), (a, 4), (about, 1), (all, 1..."
3,CKC0-MOWMqoeWf6s-szl8g,0,2016-05-28,0,MV3CcKScW05u5LVfF6ok0g,5,Lester's is located in a beautiful neighborhoo...,0,bv2nCi5Qv5vroFiqKGopiw,"[lester, s, be, locate, in, a, beautiful, neig...",4,[lester s be locate in a beautiful neighborhoo...,"[(12, 1), (1951, 1), (a, 1), (about, 1), (alon..."
4,ACFtxLv8pGrrxMm6EgjreA,0,2016-05-28,0,IXvOzsEMYtiJI0CARmj77Q,4,Love coming here. Yes the place always needs t...,0,bv2nCi5Qv5vroFiqKGopiw,"[love, come, here, yes, the, place, always, ne...",6,[love come here yes the place always need the ...,"[(1, 1), (2, 1), (a, 4), (add, 1), (always, 3)..."


In [19]:
count_lemma(df_review["lemmatized"][3000])

[(u'a', 3),
 (u'all', 2),
 (u'and', 2),
 (u'appetizer', 1),
 (u'back', 1),
 (u'be', 2),
 (u'before', 1),
 (u'birthday', 2),
 (u'care', 1),
 (u'charlene', 1),
 (u'cocktail', 1),
 (u'couple', 1),
 (u'different', 1),
 (u'drink', 1),
 (u'fantastic', 1),
 (u'for', 2),
 (u'good', 1),
 (u'great', 1),
 (u'have', 2),
 (u'house', 1),
 (u'in', 1),
 (u'just', 1),
 (u'like', 1),
 (u'make', 1),
 (u'name', 1),
 (u'new', 1),
 (u'night', 1),
 (u'of', 2),
 (u'order', 1),
 (u'perfectly', 1),
 (u'porter', 1),
 (u'pron', 8),
 (u's', 1),
 (u'say', 1),
 (u'server', 1),
 (u'sisters', 1),
 (u'stop', 1),
 (u'take', 1),
 (u'the', 3),
 (u'this', 1),
 (u'to', 1),
 (u'try', 1),
 (u'update', 1),
 (u'venue', 1)]

In [199]:
Star_5 = df_review[df_review['stars'] == 3]["lemmatized"]

#df_review[["lemmatized", "stars"]][df_review['stars'] == 5]["lemmatized"]

In [180]:
Star_5.head()

0    [love the staff , love the meat , love the pla...
1    [super simple place but amazing nonetheless . ...
2    [small unassum place that change -PRON- menu e...
3    [lester 's be locate in a beautiful neighborho...
6    [cycle pub las vegas be a blast ! get a groupo...
Name: lemmatized, dtype: object

In [200]:
# lemmat_test_1 = [",".join(df_review[]["lemmatized"][:2].sum())]
# count_lemma(lemmat_test_1)
import time
t0 = time.time()
star5_dict = count_lemma([",".join(Star_5.sum())])
t1 = time.time()
print t1-t0

2.15294599533


In [201]:
df_five_stars = pd.DataFrame(star5_dict, columns = ["word", "count"])\
.sort_values("count", ascending = False)

In [202]:
df_five_stars.head(100)

,word,count
19500,pron,153192
2688,be,93912
24723,the,84225
1476,and,47031
643,a,43328
25024,to,35970
17030,not,24975
17280,of,24453
11659,have,23926
9814,for,19405


## Find the most frequent N words in 1-star and 5-star reviews

In [96]:
A = [('happy',1),('cookie',2),('always',1),('about',4),('cannot',1)]
B = [('pasta',2),('lunch',1),('about',1),('time',4),('okay',2)]

In [104]:
# from collections import defaultdict
# d = defaultdict(list)
# for a, b in A+B:
#     d[a].append(b)

d1 = dict(A)
d1.update(dict(B))
list(d1.items())

[('about', 1),
 ('okay', 2),
 ('always', 1),
 ('lunch', 1),
 ('cannot', 1),
 ('cookie', 2),
 ('time', 4),
 ('pasta', 2),
 ('happy', 1)]

In [94]:
df_review['word_frequency'][0]

[(u'a', 3),
 (u'and', 1),
 (u'around', 1),
 (u'ask', 1),
 (u'can', 1),
 (u'cut', 1),
 (u'dinner', 1),
 (u'do', 1),
 (u'fatty', 1),
 (u'for', 1),
 (u'french', 1),
 (u'fry', 1),
 (u'get', 1),
 (u'half', 1),
 (u'hand', 1),
 (u'hot', 1),
 (u'hour', 1),
 (u'how', 1),
 (u'just', 1),
 (u'lean', 1),
 (u'line', 1),
 (u'long', 1),
 (u'love', 3),
 (u'lunch', 1),
 (u'maybe', 1),
 (u'meat', 2),
 (u'not', 2),
 (u'or', 2),
 (u'pepper', 1),
 (u'pickle', 1),
 (u'place', 1),
 (u'prepare', 1),
 (u'pron', 7),
 (u'remember', 1),
 (u'say', 1),
 (u'something', 1),
 (u'sour', 1),
 (u'staff', 1),
 (u'the', 3),
 (u'too', 2),
 (u'want', 2)]

In [205]:
df_smallset = df_review.head(1000)
df_smallset.to_pickle('yelp_review_cleaned_small.pkl')

In [206]:
## use word-to-vec to get a feature matrix and use it to 
## run linear regression